In [1]:
import face_recognition
import os
import cv2
import numpy as np

In [2]:
#!pip install face_recognition

In [3]:
#!pip install opecv-python

In [4]:
KNOWN_FACES_DIR = 'dataset'

TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = 'hog'

print('Loading known faces...')
known_faces = []
known_names = []

for filename in os.listdir(KNOWN_FACES_DIR):

        #for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):

        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{filename}')
        face_location = face_recognition.face_locations(image,model=MODEL)
        encoding = face_recognition.face_encodings(image,face_location)[0]

        known_faces.append(encoding)
        name=filename.split(".")[0]
        known_names.append(name)

Loading known faces...


In [5]:


face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()

    
    #small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    #rgb_small_frame = small_frame[:, :, ::-1]
        
    face_locations = face_recognition.face_locations(frame,model=MODEL)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_faces, face_encoding,TOLERANCE)
        name = "Unknown"
        face_distances = face_recognition.face_distance(known_faces, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_names[best_match_index]

        face_names.append(name)



    for (top, right, bottom, left), name in zip(face_locations, face_names):

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()